In [15]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [16]:
import copy
import datetime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

In [18]:
import celeri
celeri = reload(celeri)

RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
# command_file_name = './data/global/global_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
segment, station, block, mogi, sar = celeri.assign_block_labels(segment, station, block, mogi, sar)
# assembly, operators = celeri.initialize_assembly_and_operators()
# assembly = celeri.merge_geodetic_data(assembly, station, sar)
# assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
# assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)


In [25]:
def initialize_assembly_and_operators():
    """
    Initialize assembly and operators as lightweight objects
    The purpose oof this is so that we can use dot notation easily.
    """

    class Operators:
        pass

    class Assembly:
        class data:
            pass
        class sigma:
            pass
        class index:
            pass

    assembly = Assembly()
    operators = Operators()
    return assembly, operators

assembly, operators = initialize_assembly_and_operators()
assembly.data.n_stations = len(station)
type(assembly.data)



type